# **Generador de texto**

In [ ]:
import sys
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

**Cargar y preprocesar el texto**

In [ ]:
filename = "alice.txt"
with open(filename, 'r', encoding='utf-8') as file:
    raw_text = file.read()

# Preprocesar el texto y convertirlo a minusculas
raw_text = raw_text.lower()

# Crear mapeo de caracteres únicos a enteros y viceversa
chars = sorted(list(set(raw_text)))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}

# Resumen de los datos cargados
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total de caracteres: ", n_chars)
print("Tamaño del vocabulario: ", n_vocab)


**Preparar el dataset de pares entrada-salida**

In [ ]:
seq_length = 100
dataX = []
dataY = []

# Crear las secuencias de entrada y salida
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print("Total de patrones: ", n_patterns)


**Dar forma a los datos para la red LSTM**

In [ ]:
X = np.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)  # Normalizar
y = to_categorical(dataY)  # One-hot encode de la salida


**Definir el modelo LSTM**

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam')


**Configurar el callback para guardar los pesos y cargar pesos preentrenados**

In [ ]:
# Ruta para guardar los pesos
checkpoint_filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.keras"
checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

# Intentar cargar pesos preentrenados
try:
    weights_filename = "weights-improvement-19-1.9435.keras"
    model.load_weights(weights_filename)
    print(f"Pesos cargados desde: {weights_filename}")
except FileNotFoundError:
    print("Archivo de pesos no encontrado. Entrenando el modelo desde cero.")
    model.fit(X, y, epochs=20, batch_size=64, callbacks=[checkpoint])


**Generar texto a partir de una semilla aleatoria**

In [ ]:
# Seleccionar una semilla aleatoria
start = np.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
print("Semilla:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# Generar texto
print("\nGenerando texto...\n")
generated_text = []
for i in range(1000):  # Longitud del texto generado
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    generated_text.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("Texto generado:")
print("".join(generated_text))
print("\nHecho.")
